In [90]:
import torch
from torch.nn import Linear

In [91]:
b, s, k = 2, 3, 4
x = torch.rand(b, s, k) * 10
x_probs = torch.softmax(x, dim=-1)

hidden = 2

mask = torch.rand(b, s) > 0.5 # False means not masked (no noise added)

In [92]:
emb = Linear(k, hidden)
energy_head = Linear(hidden, k)

In [93]:
e = emb(x_probs)
energy = energy_head(e)

In [94]:
logits = -energy

In [95]:
logits

tensor([[[ 0.2201,  0.3261, -0.0042, -0.3876],
         [ 0.3169,  0.4383, -0.0090, -0.6329],
         [ 0.5946,  0.4470, -0.4925, -0.3150]],

        [[ 0.0827,  0.3092,  0.2162, -0.5039],
         [ 0.5259,  0.4248, -0.4029, -0.3283],
         [ 0.1030,  0.3119,  0.1840, -0.4875]]], grad_fn=<NegBackward0>)

In [96]:
x.argmax(dim=-1)

tensor([[1, 0, 2],
        [3, 2, 3]])

In [97]:
original_logits = torch.zeros_like(logits)

In [98]:
indices = x.argmax(dim=-1, keepdim=True)
original_logits.scatter_(-1, indices, 1e9)
original_logits

tensor([[[0.0000e+00, 1.0000e+09, 0.0000e+00, 0.0000e+00],
         [1.0000e+09, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 1.0000e+09, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+09],
         [0.0000e+00, 0.0000e+00, 1.0000e+09, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+09]]])

In [99]:
new_logits = torch.where(mask.unsqueeze(-1), logits, original_logits)

In [100]:
new_logits

tensor([[[ 0.0000e+00,  1.0000e+09,  0.0000e+00,  0.0000e+00],
         [ 1.0000e+09,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 5.9465e-01,  4.4700e-01, -4.9252e-01, -3.1503e-01]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+09],
         [ 5.2592e-01,  4.2478e-01, -4.0294e-01, -3.2828e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+09]]],
       grad_fn=<WhereBackward0>)

In [101]:
probs = torch.softmax(new_logits, dim=-1)

In [102]:
expected_e = (probs * energy).sum(dim=-1).mean()

In [103]:
grad = torch.autograd.grad(expected_e, new_logits, retain_graph=True)[0]

In [104]:
mask

tensor([[False, False,  True],
        [False,  True, False]])

In [105]:
grad

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  0.0000],
         [-0.0212, -0.0101,  0.0163,  0.0149]],

        [[-0.0000, -0.0000, -0.0000,  0.0000],
         [-0.0185, -0.0111,  0.0152,  0.0144],
         [-0.0000, -0.0000, -0.0000,  0.0000]]])